In [2]:
#https://gigadom.in/2020/04/18/deconstructing-convolutional-neural-networks-with-tensorflow-and-keras/
#https://keras.io/examples/vision/visualizing_what_convnets_learn/
#https://github.com/jalused/Deconvnet-keras/blob/master/Deconvnet-keras.py

import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl
import tensorflow.keras.backend as K

from tensorflow.keras.applications.vgg16 import VGG16

In [3]:
model = VGG16(weights='imagenet', include_top=True)

- We need to use tf.keras.backened.function to execute one layer at a time. This gives us access to input/output values of each layer. We need the output values of the last layer to pass into the reversing layers. Where we each layer's output values are passed the the next layer up.

In [66]:
class DLayer():
    """Base Layer for all the Reverse Layers like DConv2D, DMaxPooling2D etc.,
    
    Arguments:
    layer - A trained Layer object like Conv2D, Dense etc., 
    """
    def __init__(self, layer):
        # We don't get the weights in the base class since layers like MaxPooling2D
        # don't have weights.
        self.layer = layer
        # The below two are Keras.backend.function() which executes forward pass given the input and output layers
        # These will be set by the child classes.
        self.forward_function = None
        self.reverse_function = None
        
    def forward_pass(self, data, learning_phase=0):
        """Computes the normal forward pass of an image through the network and returns the output
        values of that layer.
        
        Arguments:
        data - The output of the previous layer i.e., layer[i+1]
        learning_phase - 
        """
        self.forward_data = self.forward_function(data)
        return self.forward_data
        
    def reverse_pass(self, data, learning_phase=0):
        """Computes the reversing of each layer. For example reversing a Conv2D layer by
        tranposing its weights(feature maps) in vertical and horizontal direction and then
        executing Conv2D on the tranposed weights.
        
        Arguments:
        """
        self.reverse_data = reverse_function(data)
        return self.reverse_data

class DInput(DLayer):
    def __init__(self, layer):
        super().__init__(layer)
    
    # overriding the parent's method
    def forward_pass(self, data, learning_phase=0):
        self.forward_data = data
        return self.forward_data
    
    def reverse_pass(self, data, learning_phase=0):
        self.reverse_data = data
        return self.reverse_data

class DConv2D(DLayer):
    """
    """
    def __init__(self, layer):
        super().__init__(layer)
        
        weights = self.layer.get_weights() # a list containing the [weights, bias]
        # weight shape = (filter_length, filter_breadth, filter_channels, # of filters)
        # bias shape = (# of filters,) since we use 1 bias per filter.
        W = weights[0]
                
        n_filters = W.shape[3]
        kernel_size = (W.shape[1], W.shape[2])
        # excluding first dimension which is batch_size; since we are gonna pass a single image only.
        inputs = tf.keras.Input(shape=layer.input_shape[1:]) 
        output = tfl.Conv2D(filters=n_filters,
                            kernel_size=kernel_size,
                            padding='same',
                            weights=weights)(inputs)
        
        self.forward_function = K.function(inputs, output)
        
        # Flip each filter horizontally and vertically
        W = np.transpose(W, (0,1,3,2))
        W = W[::-1, ::-1, :, :]
        print(W.shape)
        n_filters = W.shape[3]
        kernel_size = (W.shape[1], W.shape[2])
        b = np.zeros(n_filters)

        inputs = tf.keras.Input(shape=layer.output_shape[1:])
        output = tfl.Conv2D(filters=n_filters,
                           kernel_size=kernel_size,
                           padding='same',
                           weights=[W,b])(inputs)
        
        self.reverse_function = K.function(inputs, output)
        

class DActivation(DLayer):
    def __init__(self, layer):
        super().__init__(layer)
        
        self.activation = layer.activation
#         inputs = tfl.
                
        
        



In [65]:
def visualize_layer(model, image=None, layer_name=None, max_activation_only=False):
    """Visualize a single Layer
    """
    deconv_layers = []
    for layer in model.layers:
        print(type(layer))
        if isinstance(layer, tfl.InputLayer):
            deconv_layers.append(DInput(layer))
            
        if isinstance(layer, tfl.Conv2D):
            deconv_layers.append(DConv2D(layer))
            print(deconv_layers)
            break
    
    

visualize_layer(model)

<class 'tensorflow.python.keras.engine.input_layer.InputLayer'>
<class 'tensorflow.python.keras.layers.convolutional.Conv2D'>
(3, 64, 3, 3)
(3, 64, 3, 3)


ValueError: Layer weight shape (64, 3, 64, 3) not compatible with provided weight shape (3, 64, 3, 3)

In [35]:
x = np.array([[1,2,3],[4,5,6],[7,8,9]])
np.transpose(x, (0,1))

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])